In [1]:
import torch
import torch.nn as nn

from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt

In [14]:
class Block(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_stack = nn.Sequential(
            nn.Linear(in_features=128, out_features=128), 
            nn.ReLU()
        )
    def forward(self, x):
        return self.layer_stack(x)

class DenseNet(nn.Module):
    def __init__(self, n_blocks):
        super().__init__()
        self.bob = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=784, out_features=128),
            nn.ReLU()
        )
        self.bob2 = nn.Sequential(
            nn.Linear(in_features=128, out_features=10),
            nn.LogSoftmax(dim=1)
        )
        
        self.layer_stack = nn.Sequential()
        for b in range(n_blocks):
            self.layer_stack.append(Block())

        self.bob3 = nn.Sequential(
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Linear(in_features=2**n_blocks, out_features=128),
            nn.ReLU()
        )


    def forward(self, x):
        x = self.bob(x)
        print("Shape:", x.shape)
        for block in self.layer_stack:
            y = block(x)
            x = torch.cat((x, y), dim=0)
            print(x.shape)
        x = self.bob3(x)
        x = self.bob2(x)
        
        return x


In [15]:
mnist_train = MNIST(root="../datasets/", train=True, download=False)
mnist_test = MNIST(root="../datasets/", train=False, download=False)

X_train = mnist_train.data.type(torch.float)
Y_train = mnist_train.targets

X_test = mnist_test.data.type(torch.float)
Y_test = mnist_test.targets

In [20]:
x = X_train[:2].unsqueeze(dim=0)
print(x.shape)
model = DenseNet(5)
model(x)

torch.Size([1, 2, 28, 28])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x1568 and 784x128)

In [73]:
X_train.shape

torch.Size([60000, 28, 28])

In [71]:
from random import randint

device = torch.device("cuda")
model = DenseNet(10).to(device)

lr = 0.01

loss = nn.CrossEntropyLoss()
optim = torch.optim.SGD(params=model.parameters(), lr=lr)

BS = 32
N_B = int(60_000/BS)
for epoch in range(5):
    for batch in range(N_B):
        bi = randint(0, 60_000-BS)
        x = X_train[bi:bi+BS].to(device)
        y = Y_train[bi:bi+BS].to(device)
        out = model(x)
        l = loss(out, y)
        print(l.detach().cpu().item())
        acc = out.detach().cpu()

NameError: name 'danet' is not defined